# BAT.jl - AHMI Example

In [6]:
using BAT, ValueShapes

## Model definition to generate samples from a n-dim gaussian shell

In [7]:
struct GaussianShellDensity<:AbstractDensity
    lambda::Vector{Float64}
    r::Float64
    sigma::Float64
    dimensions::Int64
end


function BAT.density_logval(target::GaussianShellDensity, v::AbstractArray{Float64, 1})
    diff::Float64 = 0
    for i in eachindex(v)
        diff += (target.lambda[i] - v[i]) * (target.lambda[i] - v[i])
    end
    diff = sqrt(diff)
    expo::Float64 = exp(-(diff - target.r) * (diff - target.r) / (2 * target.sigma^2))
    return log(1.0 / sqrt(2 * pi * target.sigma^2) * expo)
end

dim = 2
model = GaussianShellDensity(zeros(dim), 5.0, 2.0, dim)

#define boundaries
lo_bounds = [-30.0 for i = 1:dim]
hi_bounds = [ 30.0 for i = 1:dim]
bounds = BAT.HyperRectBounds(lo_bounds, hi_bounds, BAT.reflective_bounds)

BAT.HyperRectBounds{Float64}(BAT.HyperRectVolume{Float64}([-30.0, -30.0], [30.0, 30.0]), BAT.BoundsType[BAT.reflective_bounds, BAT.reflective_bounds])

## generate samples

In [8]:
algorithm = MetropolisHastings()
n_chains = 8
n_samples = 10^5

bat_samples = bat_sample(PosteriorDensity(model, bounds), (10^5, 8), algorithm).result;

┌ Info: Trying to generate 8 viable MCMC chain(s).
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/samplers/mcmc/mcmc_tuner.jl:193
┌ Info: Selected 8 MCMC chain(s).
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/samplers/mcmc/mcmc_tuner.jl:304
┌ Info: Begin tuning of 8 MCMC chain(s).
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/samplers/mcmc/mcmc_tuner.jl:65
┌ Info: MCMC Tuning cycle 1 finished, 8 chains, 0 tuned, 8 converged.
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/samplers/mcmc/mcmc_tuner.jl:98
┌ Info: MCMC Tuning cycle 2 finished, 8 chains, 8 tuned, 8 converged.
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/samplers/mcmc/mcmc_tuner.jl:98
┌ Info: MCMC tuning of 8 chains successful after 2 cycle(s).
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/samplers/mcmc/mcmc_tuner.jl:102


## Harmonic Mean Integration
True integral value for 2D Gaussian Shell I = 31.4411
True integral value for 10D Gaussian Shell I = 1.1065e9

In [9]:
data = BAT.HMIData(bat_samples)
BAT.hm_integrate!(data)

┌ Info: Harmonic Mean Integration started. Samples in dataset 1 / 2: 	400000 / 399999	Parameters:	2
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:93
┌ Info: Data Whitening.
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:108
┌ Info: Apply Whitening Transformation to Data Set 2
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:117
┌ Info: Create Space Partitioning Tree
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:129
Progress: 100%|█████████████████████████████████████████| Time: 0:00:00
┌ Info: Determine Hyperrectangle Starting Samples
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:145
┌ Info: Determine Tolerances for Hyperrectangle Creation
└ @ BAT /home/cornelius/Projects/julia/BAT.jl/src/integration/ahmi/harmonic_mean_integration.jl:165
┌ Info

Parameters: 2	Total Samples: 799999
Data Set 1: 76 Volumes
Data Set 2: 73 Volumes

Integral Estimate (cov. weighted result):
	 31.4883  +-  0.041924


## Plotting

In [ ]:
using Plots; pyplot()
plot(data, rscale = 0.25)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*